In [1]:
import pandas as pd
import folium
import matplotlib.pyplot as plt
import preprocessing

pd.set_option('display.max_columns', 60)
pd.set_option('display.max_rows', 60)
pd.set_option('mode.chained_assignment',  None)

plt.rc('font',family='malgun gothic')

smart_card_data = pd.read_csv('./data/smart_card_data.csv')
getrouteinfoall = pd.read_csv('./data/getRouteInfoAll.csv', encoding = 'cp949', index_col = 0)
getstationbyrouteall = pd.read_csv('./data/getStationByRouteAll.csv', encoding = 'cp949', index_col = 0)
bus_sttn = pd.read_csv('./data/bus_sttn.csv')
route_curve = pd.read_csv('./data/route_curve.csv', encoding = 'cp949')

### getrouteinfoall
getrouteinfoall_df = getrouteinfoall[['ROUTE_CD','ROUTE_NO']] 

### bus_sttn
bus_sttn_df = bus_sttn[['bus_sttn_id', 'posx', 'posy']]

In [2]:
from hannam import data_preprocessing

smart_card_data = data_preprocessing.smart_card_preprocessing(smart_card_data)
smart_card_data.head(3)

,CARD_NO,TR_MEANS_TYPE,TRANSF_CNT,BUS_ROUTE_ID,BUS_CO_ID,BUS_CAR_ID,USR_TYPE,GETON_BUS_STTN_ID,GETOFF_BUS_STTN_ID,GETON_PAY,GETOFF_PAY,GETON_DATETIME,GETOFF_DATETIME,GETON_HOUR,GETOFF_HOUR
169539,734471053,675,0,30300059,130010004,130752270,1,8006006,8001930,1250,0,2022-04-25 19:19:47,2022-04-25 19:38:02,19,19
169540,661607970,676,0,30300070,130010004,130752603,1,8002794,8002155,1250,0,2022-04-25 07:53:58,2022-04-25 08:16:20,7,8
169541,706749254,675,0,30300079,130010009,130759123,1,8001721,8002802,1250,0,2022-04-25 08:07:51,2022-04-25 08:39:44,8,8


In [3]:
df = smart_card_data.copy()
df.head(3)

,CARD_NO,TR_MEANS_TYPE,TRANSF_CNT,BUS_ROUTE_ID,BUS_CO_ID,BUS_CAR_ID,USR_TYPE,GETON_BUS_STTN_ID,GETOFF_BUS_STTN_ID,GETON_PAY,GETOFF_PAY,GETON_DATETIME,GETOFF_DATETIME,GETON_HOUR,GETOFF_HOUR
169539,734471053,675,0,30300059,130010004,130752270,1,8006006,8001930,1250,0,2022-04-25 19:19:47,2022-04-25 19:38:02,19,19
169540,661607970,676,0,30300070,130010004,130752603,1,8002794,8002155,1250,0,2022-04-25 07:53:58,2022-04-25 08:16:20,7,8
169541,706749254,675,0,30300079,130010009,130759123,1,8001721,8002802,1250,0,2022-04-25 08:07:51,2022-04-25 08:39:44,8,8


## 출근 시간대 (7시, 8시, 9시)

In [4]:
from hannam import transf_data_preprocessing

df_go_to = transf_data_preprocessing.filtering_time(df, 7, 8, 9)
df_go_to.head(3) # (79278, 15)

,CARD_NO,TR_MEANS_TYPE,TRANSF_CNT,BUS_ROUTE_ID,BUS_CO_ID,BUS_CAR_ID,USR_TYPE,GETON_BUS_STTN_ID,GETOFF_BUS_STTN_ID,GETON_PAY,GETOFF_PAY,GETON_DATETIME,GETOFF_DATETIME,GETON_HOUR,GETOFF_HOUR
169540,661607970,676,0,30300070,130010004,130752603,1,8002794,8002155,1250,0,2022-04-25 07:53:58,2022-04-25 08:16:20,7,8
169541,706749254,675,0,30300079,130010009,130759123,1,8001721,8002802,1250,0,2022-04-25 08:07:51,2022-04-25 08:39:44,8,8
169545,771903902,676,0,30300037,130010006,130752044,1,8001590,8002997,1250,0,2022-04-25 08:37:12,2022-04-25 08:42:02,8,8


### 환승횟수 2회

In [5]:
transf_is_2_morning, transf_is_not_2_morning = transf_data_preprocessing.transf_2_preprocessing(df_go_to)
transf_is_2_morning.head(3)

,CARD_NO,TR_MEANS_TYPE,TRANSF_CNT,BUS_ROUTE_ID,BUS_CO_ID,BUS_CAR_ID,USR_TYPE,GETON_BUS_STTN_ID,GETOFF_BUS_STTN_ID,GETON_PAY,GETOFF_PAY,GETON_DATETIME,GETOFF_DATETIME,GETON_HOUR,GETOFF_HOUR
66,91472025,676,0,30300043,130010015,130759603,1,8001358,8001628,1250,0,2022-04-25 08:18:09,2022-04-25 09:04:33,8,9
67,91472025,675,1,30300095,130010011,130759530,1,8002935,8001250,0,0,2022-04-25 09:47:15,2022-04-25 09:52:46,9,9
68,91472025,675,2,30300067,130010001,130759786,1,8001250,8002635,0,0,2022-04-25 09:53:23,2022-04-25 10:11:20,9,10


In [6]:
transf_is_2_morning_merge = transf_data_preprocessing.transf_2_merge(transf_is_2_morning, getrouteinfoall_df, bus_sttn_df)
transf_is_2_morning_merge.head(3)

,CARD_NO,TR_MEANS_TYPE_1,ROUTE_NO_1,BUS_CO_ID_1,BUS_CAR_ID_1,TRANSF_CNT,GETON_BUS_STTN_ID_1,GETON_PAY_1,GETON_DATETIME_1,GETON_BUS_X_1,GETON_BUS_Y_1,GETOFF_BUS_STTN_ID_1,GETOFF_PAY_1,GETOFF_DATETIME_1,GETOFF_BUS_X_1,GETOFF_BUS_Y_1,TR_MEANS_TYPE_2,ROUTE_NO_2,BUS_CO_ID_2,BUS_CAR_ID_2,GETON_BUS_STTN_ID_2,GETON_PAY_2,GETON_DATETIME_2,GETON_BUS_X_2,GETON_BUS_Y_2,GETOFF_BUS_STTN_ID_2,GETOFF_PAY_2,GETOFF_DATETIME_2,GETOFF_BUS_X_2,GETOFF_BUS_Y_2,TR_MEANS_TYPE_3,ROUTE_NO_3,BUS_CO_ID_3,BUS_CAR_ID_3,GETON_BUS_STTN_ID_3,GETON_PAY_3,GETON_DATETIME_3,GETOFF_BUS_STTN_ID_3,GETOFF_PAY_3,GETOFF_DATETIME_3,GETOFF_BUS_X_3,GETOFF_BUS_Y_3
0,91472025,676,108,130010015,130759603,0,8001358,1250,2022-04-25 08:18:09,36.287640,127.464714,8001628,0,2022-04-25 09:04:33,36.349995,127.390335,675,916,130010011,130759530,8002935,0,2022-04-25 09:47:15,36.354670,127.394875,8001250,0,2022-04-25 09:52:46,36.347336,127.401566,675,514,130010001,130759786,8001250,0,2022-04-25 09:53:23,8002635,0,2022-04-25 10:11:20,36.329105,127.43359
1,302620769,676,605,130010001,130759429,0,8002486,1250,2022-04-25 08:10:00,36.335636,127.449290,8001412,0,2022-04-25 08:19:14,36.329525,127.433680,676,103,130010014,130753501,8002358,0,2022-04-25 08:26:50,36.328297,127.434410,8002734,0,2022-04-25 08:38:36,36.322990,127.414650,675,311,130010014,130753268,8002733,0,2022-04-25 09:01:22,8002635,0,2022-04-25 09:11:19,36.329105,127.43359
2,131074379,675,115,130010001,130759718,0,8005992,1250,2022-04-25 07:35:44,36.337740,127.338036,8002547,0,2022-04-25 07:56:20,36.307274,127.363945,676,201,130010008,130752121,8002547,0,2022-04-25 07:58:35,36.307274,127.363945,8002437,0,2022-04-25 08:28:33,36.329100,127.427444,675,511,130010015,130759642,8002437,0,2022-04-25 08:32:11,8002635,0,2022-04-25 08:34:53,36.329105,127.43359


### 환승횟수 1회

In [7]:
transf_is_1_morning, transf_is_not_1_morning = transf_data_preprocessing.transf_1_preprocessing(df_go_to)
transf_is_1_morning.head(3)

,CARD_NO,TR_MEANS_TYPE,TRANSF_CNT,BUS_ROUTE_ID,BUS_CO_ID,BUS_CAR_ID,USR_TYPE,GETON_BUS_STTN_ID,GETOFF_BUS_STTN_ID,GETON_PAY,GETOFF_PAY,GETON_DATETIME,GETOFF_DATETIME,GETON_HOUR,GETOFF_HOUR
7791,90027797,676,0,30300131,130010008,130752708,1,8001079,8002814,1250,0,2022-04-25 07:09:00,2022-04-25 07:36:16,7,7
7792,90027797,676,1,30300089,130010011,130759576,1,8002814,8002537,0,0,2022-04-25 07:43:02,2022-04-25 08:08:28,7,8
12975,90069429,675,0,30300055,130010014,130753212,1,8002345,8001348,1250,0,2022-04-25 08:11:11,2022-04-25 08:14:53,8,8


In [8]:
transf_is_1_morning_merge = transf_data_preprocessing.transf_1_merge(transf_is_1_morning, getrouteinfoall_df, bus_sttn_df)
transf_is_1_morning_merge.head(3)

,CARD_NO,TR_MEANS_TYPE_1,ROUTE_NO_1,BUS_CO_ID_1,BUS_CAR_ID_1,TRANSF_CNT,GETON_BUS_STTN_ID_1,GETON_PAY_1,GETON_DATETIME_1,GETON_BUS_X_1,GETON_BUS_Y_1,GETOFF_BUS_STTN_ID_1,GETOFF_PAY_1,GETOFF_DATETIME_1,GETOFF_BUS_X_1,GETOFF_BUS_Y_1,TR_MEANS_TYPE_2,ROUTE_NO_2,BUS_CO_ID_2,BUS_CAR_ID_2,GETON_BUS_STTN_ID_2,GETON_PAY_2,GETON_DATETIME_2,GETON_BUS_X_2,GETON_BUS_Y_2,GETOFF_BUS_STTN_ID_2,GETOFF_PAY_2,GETOFF_DATETIME_2,GETOFF_BUS_X_2,GETOFF_BUS_Y_2
0,90027797,676,203,130010008,130752708,0,8001079,1250,2022-04-25 07:09:00,36.30686,127.340850,8002814,0,2022-04-25 07:36:16,36.35306,127.379524,676,705,130010011,130759576,8002814,0,2022-04-25 07:43:02,36.35306,127.379524,8002537,0,2022-04-25 08:08:28,36.40083,127.40498
1,712034502,675,318,130010008,130752137,0,8001949,1250,2022-04-25 07:02:43,36.34238,127.387480,8002814,0,2022-04-25 07:07:13,36.35306,127.379524,676,705,130010011,130759536,8002814,0,2022-04-25 07:09:32,36.35306,127.379524,8002537,0,2022-04-25 07:28:37,36.40083,127.40498
2,645114698,675,211,130010002,130752376,0,8001491,1250,2022-04-25 09:03:16,36.31539,127.379196,8002814,0,2022-04-25 09:27:35,36.35306,127.379524,676,705,130010011,130759521,8002814,0,2022-04-25 09:38:16,36.35306,127.379524,8002537,0,2022-04-25 09:58:39,36.40083,127.40498


In [9]:
# 출근시간 환승 1회한 사람들 중 한남대학교 정류장이 최종 목적지인 경우
transf_1_morning_hannam = transf_is_1_morning_merge[transf_is_1_morning_merge['GETOFF_BUS_STTN_ID_2'] == 8002904]
transf_1_morning_hannam

,CARD_NO,TR_MEANS_TYPE_1,ROUTE_NO_1,BUS_CO_ID_1,BUS_CAR_ID_1,TRANSF_CNT,GETON_BUS_STTN_ID_1,GETON_PAY_1,GETON_DATETIME_1,GETON_BUS_X_1,GETON_BUS_Y_1,GETOFF_BUS_STTN_ID_1,GETOFF_PAY_1,GETOFF_DATETIME_1,GETOFF_BUS_X_1,GETOFF_BUS_Y_1,TR_MEANS_TYPE_2,ROUTE_NO_2,BUS_CO_ID_2,BUS_CAR_ID_2,GETON_BUS_STTN_ID_2,GETON_PAY_2,GETON_DATETIME_2,GETON_BUS_X_2,GETON_BUS_Y_2,GETOFF_BUS_STTN_ID_2,GETOFF_PAY_2,GETOFF_DATETIME_2,GETOFF_BUS_X_2,GETOFF_BUS_Y_2
2613,509591164,676,301,130010007,130759382,0,8001261,750,2022-04-25 09:31:31,36.331684,127.371210,8007011,0,2022-04-25 09:36:47,36.34420,127.375030,676,105,130010007,130759380,8002757,0,2022-04-25 09:43:50,36.347553,127.377150,8002904,0,2022-04-25 10:10:39,36.351395,127.42235
2614,540338972,676,301,130010007,130759376,0,8001645,1250,2022-04-25 08:28:00,36.328760,127.370575,8001089,0,2022-04-25 08:46:43,36.34730,127.375990,676,105,130010013,130759818,8002757,0,2022-04-25 08:50:31,36.347553,127.377150,8002904,0,2022-04-25 09:16:42,36.351395,127.42235
2615,763060708,675,216,130010002,130752327,0,8001645,1250,2022-04-25 08:45:45,36.328760,127.370575,8001089,0,2022-04-25 09:02:43,36.34730,127.375990,676,105,130010007,130759343,8002757,0,2022-04-25 09:07:40,36.347553,127.377150,8002904,0,2022-04-25 09:35:55,36.351395,127.42235
2616,903467790,676,301,130010007,130759381,0,8001762,1250,2022-04-25 09:10:50,36.316460,127.371350,8001089,0,2022-04-25 09:24:53,36.34730,127.375990,676,105,130010007,130759364,8002757,0,2022-04-25 09:35:37,36.347553,127.377150,8002904,0,2022-04-25 10:00:46,36.351395,127.42235
2617,771260792,676,301,130010007,130759361,0,8002831,750,2022-04-25 09:24:24,36.322803,127.370460,8001089,0,2022-04-25 09:37:00,36.34730,127.375990,676,105,130010007,130759380,8002757,0,2022-04-25 09:43:44,36.347553,127.377150,8002904,0,2022-04-25 10:10:45,36.351395,127.42235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2727,537849471,675,613,130010006,130759013,0,8002581,1250,2022-04-25 09:32:49,36.312424,127.374090,8002434,0,2022-04-25 09:46:08,36.31803,127.396645,675,314,130010004,130752202,8001940,0,2022-04-25 09:50:27,36.319010,127.400520,8002904,0,2022-04-25 10:15:23,36.351395,127.42235
2728,545722087,675,616,130010009,130759130,0,8002361,750,2022-04-25 07:44:05,36.367630,127.415760,8007274,0,2022-04-25 07:50:36,36.34656,127.417740,675,711,130010013,130759821,8007273,0,2022-04-25 07:53:18,36.346070,127.418785,8002904,0,2022-04-25 07:58:37,36.351395,127.42235
2729,743057184,676,301,130010007,130759362,0,8002976,1250,2022-04-25 08:55:02,36.304370,127.373780,8002707,0,2022-04-25 08:58:57,36.30422,127.379480,675,314,130010004,130752294,8005640,0,2022-04-25 09:04:34,36.302800,127.378940,8002904,0,2022-04-25 09:46:31,36.351395,127.42235
2730,909153966,675,614,130010008,130752172,0,8002733,1250,2022-04-25 08:29:45,36.322640,127.415110,8007271,0,2022-04-25 08:46:50,36.34344,127.422380,675,314,130010004,130752291,8007271,0,2022-04-25 09:01:30,36.343440,127.422380,8002904,0,2022-04-25 09:04:36,36.351395,127.42235


In [12]:
def station_route_merge(getstationbyrouteall,route_cd):
    exdata = getstationbyrouteall[getstationbyrouteall['ROUTE_CD']==route_cd].reset_index()
    index_condition = exdata[exdata['BUSSTOP_TP'] == '2'].index.max() -1
    exdata = exdata.iloc[:index_condition]
    return exdata

In [36]:
# 314번 버스
route_314 = station_route_merge(getstationbyrouteall,30300058)
# 711번 버스
route_711 = station_route_merge(getstationbyrouteall,30300091)
# 105번 버스
route_105 = station_route_merge(getstationbyrouteall,30300040)

In [28]:
import pydeck as pdk


# 출발지를 표시하는 레이어
layer_on = pdk.Layer(
    'ScatterplotLayer',
    data=transf_1_morning_hannam,
    get_position='[GETON_BUS_Y_1, GETON_BUS_X_1]',
    get_radius=100,
    get_fill_color=[255, 0, 0],  # 포인트 색상 (RGB 값)
    pickable=True
)

# 환승지를 표시하는 레이어
layer_mid = pdk.Layer(
    'ScatterplotLayer',
    data=transf_1_morning_hannam,
    get_position='[GETON_BUS_Y_2, GETON_BUS_X_2]',
    get_radius=100,
    get_fill_color=[255, 150, 0],  # 주황색
    pickable=True
)

# 도착지를 표시하는 레이어
layer_off = pdk.Layer(
    'ScatterplotLayer',
    data=transf_1_morning_hannam,
    get_position='[GETOFF_BUS_Y_2, GETOFF_BUS_X_2]',
    get_radius=100,
    get_fill_color=[255,200, 0],  # 노란색
    pickable=True
)

# 지도 객체 생성
view_state = pdk.ViewState(latitude=36.3504119, longitude=127.3845475, zoom=11)
r = pdk.Deck(layers=[layer_on, layer_mid, layer_off], initial_view_state=view_state)

# 지도 출력
# r.to_html('map.html')
r

{
  "initialViewState": {
    "latitude": 36.3504119,
    "longitude": 127.3845475,
    "zoom": 11
  },
  "layers": [
    {
      "@@type": "ScatterplotLayer",
      "data": [
        {
          "BUS_CAR_ID_1": 130759382,
          "BUS_CAR_ID_2": 130759380,
          "BUS_CO_ID_1": 130010007,
          "BUS_CO_ID_2": 130010007,
          "CARD_NO": 509591164,
          "GETOFF_BUS_STTN_ID_1": 8007011,
          "GETOFF_BUS_STTN_ID_2": 8002904,
          "GETOFF_BUS_X_1": 36.3442,
          "GETOFF_BUS_X_2": 36.351395,
          "GETOFF_BUS_Y_1": 127.37503,
          "GETOFF_BUS_Y_2": 127.42235,
          "GETOFF_DATETIME_1": {},
          "GETOFF_DATETIME_2": {},
          "GETOFF_PAY_1": 0,
          "GETOFF_PAY_2": 0,
          "GETON_BUS_STTN_ID_1": 8001261,
          "GETON_BUS_STTN_ID_2": 8002757,
          "GETON_BUS_X_1": 36.331684,
          "GETON_BUS_X_2": 36.347553,
          "GETON_BUS_Y_1": 127.37121,
          "GETON_BUS_Y_2": 127.37715,
          "GETON_DATETIME_1": {},
          "GETON_DATETIME_2": {},
          "GETON_PAY_1": 750,
          "GETON_PAY_2": 0,
          "ROUTE_NO_1": "301",
          "ROUTE_NO_2": "105",
          "TRANSF_CNT": 0,
          "TR_MEANS_TYPE_1": 676,
          "TR_MEANS_TYPE_2": 676
        },
        {
          "BUS_CAR_ID_1": 130759376,
          "BUS_CAR_ID_2": 130759818,
          "BUS_CO_ID_1": 130010007,
          "BUS_CO_ID_2": 130010013,
          "CARD_NO": 540338972,
          "GETOFF_BUS_STTN_ID_1": 8001089,
          "GETOFF_BUS_STTN_ID_2": 8002904,
          "GETOFF_BUS_X_1": 36.3473,
          "GETOFF_BUS_X_2": 36.351395,
          "GETOFF_BUS_Y_1": 127.37599,
          "GETOFF_BUS_Y_2": 127.42235,
          "GETOFF_DATETIME_1": {},
          "GETOFF_DATETIME_2": {},
          "GETOFF_PAY_1": 0,
          "GETOFF_PAY_2": 0,
          "GETON_BUS_STTN_ID_1": 8001645,
          "GETON_BUS_STTN_ID_2": 8002757,
          "GETON_BUS_X_1": 36.32876,
          "GETON_BUS_X_2": 36.347553,
          "GETON_BUS_Y_1": 127.370575,
          "GETON_BUS_Y_2": 127.37715,
          "GETON_DATETIME_1": {},
          "GETON_DATETIME_2": {},
          "GETON_PAY_1": 1250,
          "GETON_PAY_2": 0,
          "ROUTE_NO_1": "301",
          "ROUTE_NO_2": "105",
          "TRANSF_CNT": 0,
          "TR_MEANS_TYPE_1": 676,
          "TR_MEANS_TYPE_2": 676
        },
        {
          "BUS_CAR_ID_1": 130752327,
          "BUS_CAR_ID_2": 130759343,
          "BUS_CO_ID_1": 130010002,
          "BUS_CO_ID_2": 130010007,
          "CARD_NO": 763060708,
          "GETOFF_BUS_STTN_ID_1": 8001089,
          "GETOFF_BUS_STTN_ID_2": 8002904,
          "GETOFF_BUS_X_1": 36.3473,
          "GETOFF_BUS_X_2": 36.351395,
          "GETOFF_BUS_Y_1": 127.37599,
          "GETOFF_BUS_Y_2": 127.42235,
          "GETOFF_DATETIME_1": {},
          "GETOFF_DATETIME_2": {},
          "GETOFF_PAY_1": 0,
          "GETOFF_PAY_2": 0,
          "GETON_BUS_STTN_ID_1": 8001645,
          "GETON_BUS_STTN_ID_2": 8002757,
          "GETON_BUS_X_1": 36.32876,
          "GETON_BUS_X_2": 36.347553,
          "GETON_BUS_Y_1": 127.370575,
          "GETON_BUS_Y_2": 127.37715,
          "GETON_DATETIME_1": {},
          "GETON_DATETIME_2": {},
          "GETON_PAY_1": 1250,
          "GETON_PAY_2": 0,
          "ROUTE_NO_1": "216",
          "ROUTE_NO_2": "105",
          "TRANSF_CNT": 0,
          "TR_MEANS_TYPE_1": 675,
          "TR_MEANS_TYPE_2": 676
        },
        {
          "BUS_CAR_ID_1": 130759381,
          "BUS_CAR_ID_2": 130759364,
          "BUS_CO_ID_1": 130010007,
          "BUS_CO_ID_2": 130010007,
          "CARD_NO": 903467790,
          "GETOFF_BUS_STTN_ID_1": 8001089,
          "GETOFF_BUS_STTN_ID_2": 8002904,
          "GETOFF_BUS_X_1": 36.3473,
          "GETOFF_BUS_X_2": 36.351395,
          "GETOFF_BUS_Y_1": 127.37599,
          "GETOFF_BUS_Y_2": 127.42235,
          "GETOFF_DATETIME_1": {},
          "GETOFF_DATETIME_2": {},
          "GETOFF_PAY_1": 0,
          "GETOFF_PAY_2": 0,
          "

In [31]:
import pydeck as pdk


# 출발지를 표시하는 레이어
layer_on = pdk.Layer(
    'ScatterplotLayer',
    data=transf_1_morning_hannam,
    get_position='[GETON_BUS_Y_1, GETON_BUS_X_1]',
    get_radius=100,
    get_fill_color=[255, 0, 0],  # 포인트 색상 (RGB 값)
    pickable=True
)

# 환승지를 표시하는 레이어
layer_mid = pdk.Layer(
    'ScatterplotLayer',
    data=transf_1_morning_hannam,
    get_position='[GETON_BUS_Y_2, GETON_BUS_X_2]',
    get_radius=100,
    get_fill_color=[255, 150, 0],  # 주황색
    pickable=True
)

# 도착지를 표시하는 레이어
layer_off = pdk.Layer(
    'ScatterplotLayer',
    data=transf_1_morning_hannam,
    get_position='[GETOFF_BUS_Y_2, GETOFF_BUS_X_2]',
    get_radius=100,
    get_fill_color=[255,200, 0],  # 노란색
    pickable=True
)

layer_314 = pdk.Layer(
    "ScatterplotLayer",
    data=route_314,  # 시각화할 데이터프레임
    get_position=["GPS_LONG", "GPS_LATI"],  # 경도와 위도를 지정
    get_radius=100,  # 포인트 반경 크기
    get_fill_color=[0, 100, 50],   # 초록색
    pickable=True, 
)

layer_711 = pdk.Layer(
    "ScatterplotLayer",
    data=route_711,  # 시각화할 데이터프레임
    get_position=["GPS_LONG", "GPS_LATI"],  # 경도와 위도를 지정
    get_radius=100,  # 포인트 반경 크기
    get_fill_color=[0, 50, 200],  # 파란색
    pickable=True, 
)

layer_105 = pdk.Layer(
    "ScatterplotLayer",
    data=route_105,  # 시각화할 데이터프레임
    get_position=["GPS_LONG", "GPS_LATI"],  # 경도와 위도를 지정
    get_radius=100,  # 포인트 반경 크기
    get_fill_color=[100, 0, 150],  # 보라색
    pickable=True, 
)

# 지도 객체 생성
view_state = pdk.ViewState(latitude=36.3504119, longitude=127.3845475, zoom=11)
r = pdk.Deck(layers=[layer_on, layer_mid,layer_off, layer_314,layer_105, layer_711], initial_view_state=view_state)

# 지도 출력
# r.to_html('map.html')
r

{
  "initialViewState": {
    "latitude": 36.3504119,
    "longitude": 127.3845475,
    "zoom": 11
  },
  "layers": [
    {
      "@@type": "ScatterplotLayer",
      "data": [
        {
          "BUS_CAR_ID_1": 130759382,
          "BUS_CAR_ID_2": 130759380,
          "BUS_CO_ID_1": 130010007,
          "BUS_CO_ID_2": 130010007,
          "CARD_NO": 509591164,
          "GETOFF_BUS_STTN_ID_1": 8007011,
          "GETOFF_BUS_STTN_ID_2": 8002904,
          "GETOFF_BUS_X_1": 36.3442,
          "GETOFF_BUS_X_2": 36.351395,
          "GETOFF_BUS_Y_1": 127.37503,
          "GETOFF_BUS_Y_2": 127.42235,
          "GETOFF_DATETIME_1": {},
          "GETOFF_DATETIME_2": {},
          "GETOFF_PAY_1": 0,
          "GETOFF_PAY_2": 0,
          "GETON_BUS_STTN_ID_1": 8001261,
          "GETON_BUS_STTN_ID_2": 8002757,
          "GETON_BUS_X_1": 36.331684,
          "GETON_BUS_X_2": 36.347553,
          "GETON_BUS_Y_1": 127.37121,
          "GETON_BUS_Y_2": 127.37715,
          "GETON_DATETIME_1": {},
          "GETON_DATETIME_2": {},
          "GETON_PAY_1": 750,
          "GETON_PAY_2": 0,
          "ROUTE_NO_1": "301",
          "ROUTE_NO_2": "105",
          "TRANSF_CNT": 0,
          "TR_MEANS_TYPE_1": 676,
          "TR_MEANS_TYPE_2": 676
        },
        {
          "BUS_CAR_ID_1": 130759376,
          "BUS_CAR_ID_2": 130759818,
          "BUS_CO_ID_1": 130010007,
          "BUS_CO_ID_2": 130010013,
          "CARD_NO": 540338972,
          "GETOFF_BUS_STTN_ID_1": 8001089,
          "GETOFF_BUS_STTN_ID_2": 8002904,
          "GETOFF_BUS_X_1": 36.3473,
          "GETOFF_BUS_X_2": 36.351395,
          "GETOFF_BUS_Y_1": 127.37599,
          "GETOFF_BUS_Y_2": 127.42235,
          "GETOFF_DATETIME_1": {},
          "GETOFF_DATETIME_2": {},
          "GETOFF_PAY_1": 0,
          "GETOFF_PAY_2": 0,
          "GETON_BUS_STTN_ID_1": 8001645,
          "GETON_BUS_STTN_ID_2": 8002757,
          "GETON_BUS_X_1": 36.32876,
          "GETON_BUS_X_2": 36.347553,
          "GETON_BUS_Y_1": 127.370575,
          "GETON_BUS_Y_2": 127.37715,
          "GETON_DATETIME_1": {},
          "GETON_DATETIME_2": {},
          "GETON_PAY_1": 1250,
          "GETON_PAY_2": 0,
          "ROUTE_NO_1": "301",
          "ROUTE_NO_2": "105",
          "TRANSF_CNT": 0,
          "TR_MEANS_TYPE_1": 676,
          "TR_MEANS_TYPE_2": 676
        },
        {
          "BUS_CAR_ID_1": 130752327,
          "BUS_CAR_ID_2": 130759343,
          "BUS_CO_ID_1": 130010002,
          "BUS_CO_ID_2": 130010007,
          "CARD_NO": 763060708,
          "GETOFF_BUS_STTN_ID_1": 8001089,
          "GETOFF_BUS_STTN_ID_2": 8002904,
          "GETOFF_BUS_X_1": 36.3473,
          "GETOFF_BUS_X_2": 36.351395,
          "GETOFF_BUS_Y_1": 127.37599,
          "GETOFF_BUS_Y_2": 127.42235,
          "GETOFF_DATETIME_1": {},
          "GETOFF_DATETIME_2": {},
          "GETOFF_PAY_1": 0,
          "GETOFF_PAY_2": 0,
          "GETON_BUS_STTN_ID_1": 8001645,
          "GETON_BUS_STTN_ID_2": 8002757,
          "GETON_BUS_X_1": 36.32876,
          "GETON_BUS_X_2": 36.347553,
          "GETON_BUS_Y_1": 127.370575,
          "GETON_BUS_Y_2": 127.37715,
          "GETON_DATETIME_1": {},
          "GETON_DATETIME_2": {},
          "GETON_PAY_1": 1250,
          "GETON_PAY_2": 0,
          "ROUTE_NO_1": "216",
          "ROUTE_NO_2": "105",
          "TRANSF_CNT": 0,
          "TR_MEANS_TYPE_1": 675,
          "TR_MEANS_TYPE_2": 676
        },
        {
          "BUS_CAR_ID_1": 130759381,
          "BUS_CAR_ID_2": 130759364,
          "BUS_CO_ID_1": 130010007,
          "BUS_CO_ID_2": 130010007,
          "CARD_NO": 903467790,
          "GETOFF_BUS_STTN_ID_1": 8001089,
          "GETOFF_BUS_STTN_ID_2": 8002904,
          "GETOFF_BUS_X_1": 36.3473,
          "GETOFF_BUS_X_2": 36.351395,
          "GETOFF_BUS_Y_1": 127.37599,
          "GETOFF_BUS_Y_2": 127.42235,
          "GETOFF_DATETIME_1": {},
          "GETOFF_DATETIME_2": {},
          "GETOFF_PAY_1": 0,
          "GETOFF_PAY_2": 0,
          "

In [77]:
import pydeck as pdk



route_314 = getstationbyrouteall[getstationbyrouteall['ROUTE_CD']==30300058]
layer_314 = pdk.Layer(
    "ScatterplotLayer",
    data=route_314,  # 시각화할 데이터프레임
    get_position=["GPS_LONG", "GPS_LATI"],  # 경도와 위도를 지정
    get_radius=50,  # 포인트 반경 크기
    get_fill_color=[0, 100, 50],   # 초록색
    pickable=True, 
)



# 지도 객체 생성
view_state = pdk.ViewState(latitude=36.3504119, longitude=127.3845475, zoom=11)
r = pdk.Deck(layers=[layer_314], initial_view_state=view_state)

# 지도 출력
# r.to_html('map.html')
r

{
  "initialViewState": {
    "latitude": 36.3504119,
    "longitude": 127.3845475,
    "zoom": 11
  },
  "layers": [
    {
      "@@type": "ScatterplotLayer",
      "data": [
        {
          "BUSSTOP_ENG_NM": "---",
          "BUSSTOP_NM": "\uc717\uc0ac\uc815\uc0bc\uac70\ub9ac",
          "BUSSTOP_SEQ": 1,
          "BUSSTOP_TP": "1",
          "BUS_NODE_ID": 9007040,
          "BUS_STOP_ID": 9007040,
          "GPS_LATI": 36.293003,
          "GPS_LONG": 127.399086,
          "ROAD_NM": " ",
          "ROAD_NM_ADDR": " ",
          "ROUTE_CD": 30300058,
          "TOTAL_DIST": 71
        },
        {
          "BUSSTOP_ENG_NM": "Witsajeong-samgeori ",
          "BUSSTOP_NM": "\uc717\uc0ac\uc815\uc0bc\uac70\ub9ac",
          "BUSSTOP_SEQ": 2,
          "BUSSTOP_TP": " ",
          "BUS_NODE_ID": 8007040,
          "BUS_STOP_ID": 23400,
          "GPS_LATI": 36.29278,
          "GPS_LONG": 127.397995,
          "ROAD_NM": " ",
          "ROAD_NM_ADDR": " ",
          "ROUTE_CD": 30300058,
          "TOTAL_DIST": 177
        },
        {
          "BUSSTOP_ENG_NM": "Zoo Land",
          "BUSSTOP_NM": "\uc624\uc6d4\ub4dc",
          "BUSSTOP_SEQ": 3,
          "BUSSTOP_TP": " ",
          "BUS_NODE_ID": 8001393,
          "BUS_STOP_ID": 22650,
          "GPS_LATI": 36.290104,
          "GPS_LONG": 127.39497,
          "ROAD_NM": "\uc0ac\uc815\uacf5\uc6d0\ub85c",
          "ROAD_NM_ADDR": " ",
          "ROUTE_CD": 30300058,
          "TOTAL_DIST": 609
        },
        {
          "BUSSTOP_ENG_NM": "Zoo Land Entrance",
          "BUSSTOP_NM": "\uc624\uc6d4\ub4dc\ud68c\uc804\uad50\ucc28\ub85c",
          "BUSSTOP_SEQ": 4,
          "BUSSTOP_TP": " ",
          "BUS_NODE_ID": 8001523,
          "BUS_STOP_ID": 22620,
          "GPS_LATI": 36.29127,
          "GPS_LONG": 127.38825,
          "ROAD_NM": "\uc0b0\uc11c\ub85c",
          "ROAD_NM_ADDR": " ",
          "ROUTE_CD": 30300058,
          "TOTAL_DIST": 1333
        },
        {
          "BUSSTOP_ENG_NM": "Sangdang",
          "BUSSTOP_NM": "\uc0c1\ub2f9",
          "BUSSTOP_SEQ": 5,
          "BUSSTOP_TP": " ",
          "BUS_NODE_ID": 8001895,
          "BUS_STOP_ID": 22640,
          "GPS_LATI": 36.294456,
          "GPS_LONG": 127.38949,
          "ROAD_NM": "\uc0b0\uc11c\ub85c",
          "ROAD_NM_ADDR": " ",
          "ROUTE_CD": 30300058,
          "TOTAL_DIST": 1704
        },
        {
          "BUSSTOP_ENG_NM": "Daejeon Lifelong Education Center(Jangteogol)",
          "BUSSTOP_NM": "\uc0b0\uc131\uc5b4\ub9b0\uc774\ub3c4\uc11c\uad00",
          "BUSSTOP_SEQ": 6,
          "BUSSTOP_TP": " ",
          "BUS_NODE_ID": 8001437,
          "BUS_STOP_ID": 22600,
          "GPS_LATI": 36.295925,
          "GPS_LONG": 127.38716,
          "ROAD_NM": "\uc0b0\uc11c\ub85c",
          "ROAD_NM_ADDR": " ",
          "ROUTE_CD": 30300058,
          "TOTAL_DIST": 2057
        },
        {
          "BUSSTOP_ENG_NM": "Sajeonggyo(Bridge)",
          "BUSSTOP_NM": "\uc0ac\uc815\uad50",
          "BUSSTOP_SEQ": 7,
          "BUSSTOP_TP": " ",
          "BUS_NODE_ID": 8005915,
          "BUS_STOP_ID": 22530,
          "GPS_LATI": 36.295982,
          "GPS_LONG": 127.38366,
          "ROAD_NM": " ",
          "ROAD_NM_ADDR": " ",
          "ROUTE_CD": 30300058,
          "TOTAL_DIST": 2373
        },
        {
          "BUSSTOP_ENG_NM": "Boksu-dong Sajeonggyo(Bridge)",
          "BUSSTOP_NM": "\ubcf5\uc218\ub3d9\uc0ac\uc815\uad50",
          "BUSSTOP_SEQ": 8,
          "BUSSTOP_TP": " ",
          "BUS_NODE_ID": 8005914,
          "BUS_STOP_ID": 30150,
          "GPS_LATI": 36.297802,
          "GPS_LONG": 127.37948,
          "ROAD_NM": " ",
          "ROAD_NM_ADDR": " ",
          "ROUTE_CD": 30300058,
          "TOTAL_DIST": 2814
        },
        {
          "BUSSTOP_ENG_NM": "In Boksu-gil",
          "BUSSTOP_NM": "\uc548\ubcf5\uc218\uae38",
          "BUSSTOP_SEQ": 9,
          "BUSSTOP_TP": " ",
          "BUS_NODE_ID": 8005934,
          "BUS_STOP_ID": 23170,
          "GP

In [92]:
import geopandas as gpd

daejeon_file = './data/LARD_ADM_SECT_SGG_대전/LARD_ADM_SECT_SGG_30.shp'
daejeon = gpd.read_file(daejeon_file, encoding = 'cp949')
daejeon

,ADM_SECT_C,SGG_NM,SGG_OID,COL_ADM_SE,GID,geometry
0,30230,대덕구,NaN,30230,165,"POLYGON ((998333.118 1831051.084, 998569.440 1..."
1,30200,유성구,NaN,30200,166,"POLYGON ((986874.610 1814985.122, 986851.599 1..."
2,30170,서구,NaN,30170,167,"POLYGON ((988338.682 1810216.004, 988338.685 1..."
3,30140,중구,NaN,30140,168,"POLYGON ((995945.175 1811588.887, 995945.178 1..."
4,30110,동구,NaN,30110,169,"POLYGON ((999321.491 1826867.967, 999239.953 1..."


In [119]:
# daejeon = gpd.read_file('./data/TL_SCCO_SIG.json')
# SIG_CD_DJ = ['30230','30110','30170','30200','30140']
# daejeon_sgg = daejeon[daejeon['SIG_CD'].isin(SIG_CD_DJ)]
# folium.Choropleth(geo_data=daejeon_sgg['geometry']).add_to(m)
# m

In [121]:
import folium

# 주어진 데이터프레임에서 경로 정보 추출
route_314 = station_route_merge(getstationbyrouteall,30300058)

# 지도 중심 위치 설정
m = folium.Map(location=[route_314['GPS_LATI'].mean(), route_314['GPS_LONG'].mean()], tiles='cartodbdark_matter', zoom_start=12)

# LineString으로 좌표들을 이어서 표시
coordinates = [(row['GPS_LATI'], row['GPS_LONG']) for _, row in route_314.iterrows()]
folium.PolyLine(locations=coordinates, color='red').add_to(m)
# folium.Choropleth(geo_data=daejeon_sgg['geometry']).add_to(m)
m